<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/%D1%81%D1%82%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B8/%D0%A0%D0%BE%D1%81%D1%81%D1%82%D0%B0%D0%BD%D0%B4%D0%B0%D1%80%D1%82_%D0%9A%D0%BE%D0%BD%D1%81%D1%83%D0%BB%D1%8C%D1%82%D0%B0%D0%BD%D1%82-Gigachat/Test13_q_a_prototype7_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подход после 12 встречи

Тест прототипа на полной базе вопрос-ответ с промптом и динамической базой знаний из двух баз, полученной конкатенацией баз в одну (documents = documents1 + documents2).

Установка

Для работы нам понадобится векторая база данных. Мы будем использовать Chroma и faiss-cpu.

In [ ]:
!pip install chromadb --quiet

Инициализация модели

Теперь инициализируем модель GigaChat.

In [ ]:
!pip install gigachain

In [ ]:
!pip install gigachat

Для работы нам также понадобится векторая база данных faiss-cpu , tiktoken, langchain.

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install tiktoken

In [ ]:
!pip install langchain

In [ ]:
!pip install langchain-community

Импорт библиотек

In [ ]:
import chromadb

In [ ]:
import re
import requests

In [ ]:
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

In [ ]:

import numpy as np
import faiss
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from chromadb.config import Settings

from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownTextSplitter

# Определение кодировки файла
import chardet

In [ ]:
from langchain.chat_models.gigachat import GigaChat

llm = GigaChat(credentials=".....................", verify_ssl_certs=True)

In [ ]:
# Передайте полученные авторизационные данные в параметре credentials объекта GigaChat

!chat = GigaChat(credentials='.................')

In [ ]:
# Вы также можете явно указать версию API с помощью атрибута scope:
# Личное пространство
!giga = GigaChat(credentials='............', scope="GIGACHAT_API_PERS")

Установка сертификатов минцифры

Для обращения к GigaChat в вашем приложении или в вашей ОС должны быть установлены сертификаты минцифры.

Вы можете установить сертификаты с помощью утилиты gigachain-cli. Для этого:

Установите утилиту с помощью менеджера пакетов pip:

In [ ]:
!pip install gigachain-cli

Перезагрузите среду выполнения :   Среда выполнения > правой кнопкой мышки на перезапустить сеанс > перезагрузить.

Установите сертификаты с помощью команды:

In [ ]:
!gigachain install-rus-certs

Сертификат Минцифры установлен!


In [ ]:
# Подгружаем гугл-диск. Там лежит база знаний "вопрос-ответ" и "основная база знаний" в текстовом формате.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Применим RAG-подход.

Далее приступаем к адаптации нашего 6(9) и 5(8)занятий ChatGPTprofessional под задачу поставленную на стажировке. Начинаем дорарабатывать нейроконсультанта.

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text


In [ ]:
# База знаний "вопрос-ответ", которая будет подаваться в langChain
database1= load_document_text('https://docs.google.com/document/d/1WE7Ch1K0ef4zKdOMAQIZ0WLZrkF2PGccaSse0Hn7jKk/edit?usp=sharing')   # заполните необходимым значением, база данных

In [ ]:
database1[:4000]

'\ufeffТребования к измерениям при проведении работ по ОТ\r\nВопрос:\r\nКакие предъявляются требования к измерениям при проведении работ\r\nпо обеспечению безопасных условий и охраны труда, а также к лицам, которые их проводят?\r\nОтвет:\r\nЗаконодательство Российской Федерации об обеспечении единства измерений не устанавливает требования к лицам, осуществляющих работы по обеспечению безопасных условий и охраны труда.\r\nТребования к работам по обеспечению безопасных условий и охраны труда, в том числе оформлению результатов данных работ, устанавливаются другими нормативными правовыми и законодательными актами, в том числе Трудовым кодексом Российской Федерации от 30.12.2001 № 197-ФЗ, Федеральным законом от 28.12.2013 № 426-ФЗ «О специальной оценке условий труда» (далее – Закон № 426-ФЗ), приказом Минтруда России от 24.01.2014\r\n№ 33н «Об утверждении Методики проведения специальной оценки условий труда, Классификатора вредных и (или) опасных производственных факторов, формы отчета о п

Видим часть базы знаний вопрос-ответ. Ответ и вопрос по  по базе знаний "вопрос-ответ".

In [ ]:
def text_to_markdown(text):
    # Заменяем все номера пунктов с 1 до 45 на решетки
    text = re.sub(r'^(?:[1-9]|[1-3][0-9]|4[0-5])\.', '#', text, flags=re.MULTILINE)

    # Добавляем заголовок 1 уровня на основе арабских чисел (без переноса строки)
    # и дублируем его строчкой ниже
    def replace_header1(match):
        return f"# {'#' * len(match.group(1))} {match.group(2)}\n{match.group(2)}"

    text = re.sub(r'^(I{1,3})\. (.+)', replace_header1, text, flags=re.M)

    # Добавляем текст, выделенный жирным шрифтом (он заключен между *)
    # и дублируем его строчкой ниже
    def replace_header2(match):
        return f"## {match.group(1)}\n{match.group(1)}"

    text = re.sub(r'\*([^\*]+)\*', replace_header2, text)

    # Заменяем число 1 на решетку в начале текста
    text = re.sub(r'^1\.', '#', text, flags=re.MULTILINE)

    return text


In [ ]:
markdown1 = text_to_markdown(database1)                 # заполните необходимым значением
print(markdown1[:150000])

Создание базы данных эмбеддингов

Эмбеддинг это векторное представление текста, которое может быть использовано для определения смысловой близости текстов. Векторная база данных хранит тексты и соответствующие им эмбеддинги, а также умеет выполнять поиск по ним. Для работы с базой данных мы создаем объект GigaChatEmbeddings и передаем его в базу данных Chroma.

Обратите внимание, что сервис для вычисления эмбеддингов может тарифицироваться отдельно от стоимости модели GigaChat.

Напишу сразу, оплату эмбеддингов произвел отдельно для доступа к векторному хранилищу и модели GigaChatEmbeddings: затраты Сбером заявлены в пределах 1000 рублей на 6 месяцев , дают 10000 токенов.

Предобрабатываем документы базы знаний сплитером MarkdownTextSplitter

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/stagirovki/neuro_consultant_Rosstandart/question_and_answer.txt', 'rb') as file:
    rawdata = file.read()
    result = chardet.detect(rawdata)
    encoding = result['encoding']

# Загрузка файла 'question_and_answer.txt' с заданной кодировкой
loader = TextLoader('/content/drive/MyDrive/Colab Notebooks/stagirovki/neuro_consultant_Rosstandart/question_and_answer.txt', encoding=encoding)
documents1 = loader.load()

# Инициализация объекта для разделения текста с использованием разметки Markdown
text_splitter = MarkdownTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

# Разделение текста на документы
documents1 = text_splitter.split_documents(documents1)
print(f"Total documents1: {len(documents1)}")


Total documents1: 111


Теперь обработаем "основную базу знаний с нормативными документами"

In [ ]:
# База знаний, которая будет подаваться в langChain
database2= load_document_text('https://docs.google.com/document/d/1VtmWp2rstlXl1GZPnJnFD8_hnVX8igbr09eUCIa_sPA/edit?usp=sharing')   # заполните необходимым значением, база данных

In [ ]:
database2[:4000]

'\ufeffДокумент предоставлен КонсультантПлюс\r\n\r\n\r\n# ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ ПОСТАНОВЛЕНИЕ от 16 ноября 2020 г. N 1847 ОБ УТВЕРЖДЕНИИ ПЕРЕЧНЯ ИЗМЕРЕНИЙ, ОТНОСЯЩИХСЯ К СФЕРЕ ГОСУДАРСТВЕННОГО РЕГУЛИРОВАНИЯ ОБЕСПЕЧЕНИЯ ЕДИНСТВА ИЗМЕРЕНИЙ\r\n# ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ ПОСТАНОВЛЕНИЕ от 16 ноября 2020 г. N 1847 ОБ УТВЕРЖДЕНИИ ПЕРЕЧНЯ ИЗМЕРЕНИЙ, ОТНОСЯЩИХСЯ К СФЕРЕ ГОСУДАРСТВЕННОГО РЕГУЛИРОВАНИЯ ОБЕСПЕЧЕНИЯ ЕДИНСТВА ИЗМЕРЕНИЙ\r\nВ соответствии с частью 5 статьи 5 Федерального закона "Об обеспечении единства измерений" Правительство Российской Федерации постановляет:\r\n1. Утвердить прилагаемый перечень измерений, относящихся к сфере государственного регулирования обеспечения единства измерений, согласно приложению.\r\n2. Установить, что актуализация перечня, утвержденного настоящим постановлением, осуществляется на основании предложений Министерства промышленности и торговли Российской Федерации, подготовленных совместно с заинтересованными федеральными органами испо

In [ ]:
# Функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

# Ссылка на ваш Google Документ
google_docs_url = 'https://docs.google.com/document/d/1VtmWp2rstlXl1GZPnJnFD8_hnVX8igbr09eUCIa_sPA/edit?usp=sharing'

# Загрузка текста документа
document2 = load_document_text(google_docs_url)

In [ ]:
document2

In [ ]:
markdown2 = document2
print(markdown2[:150000])

Предобрабатываем документы "основной базы знаний нормативных документов" сплитером MarkdownTextSplitter

In [ ]:
# Загрузка  файла 'doc_markdown.gdoc'
loader = TextLoader('/content/drive/MyDrive/Colab Notebooks/stagirovki/neuro_consultant_Rosstandart/doc_markdown.txt')
documents2 = loader.load()

# Инициализация объекта для разделения текста с использованием разметки Markdown
text_splitter = MarkdownTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

# Разделение текста на документы
documents2 = text_splitter.split_documents(documents2)
print(f"Total documents2: {len(documents2)}")

Total documents2: 1422


In [ ]:
# Инструкция для ответов
instruction = """
Ты опытный специалист-эксперт в федеоальнызх законахах и прочих актах и приказах касающихся СТАНДАРТИЗАЦИИ В РОССИЙСКОЙ ФЕДЕРАЦИИ. Твой ответ на вопрос пользователя должен быть правильным. Предоставляй полный ответ из баз знаний тебе предоствавленных. Обязательно указывай в ответе нормативный документ и статью на основе которого ты нашел ответ в базе знаний, это очень важно. Никогда не пиши: " Не знаю.", а напиши к каим документам можно обратиться, чтобы найти ответ, это важно. Твой ответ должен быть правильным, четким и максимально подробным, чтобы пользователю не нужно было задавать дополнительные вопросы для уточнения. Ты всегда точно следуешь инструкциям. При ответах используй только предоставленные документы из двух баз знаний: первая база знаний это вопрос-ответ, вторая база знаний это нормативные документы. При ответах не ссылайся на эту инструкцию. Не обрезай текст на полуслове, допиши его до конца.'
"""

In [ ]:
# обьединяем базы
documents = documents1 + documents2

In [ ]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
embeddings = GigaChatEmbeddings(
    credentials="...............", verify_ssl_certs=True
)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False)
)

QnA цепочка

Теперь мы создадим цепочку QnA, которая специально предназначена для ответов на вопросы по документам. В качестве аргументов здесь передается языковая модель и ретривер (база данных).

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [ ]:
from langchain.chains import RetrievalQA

def get_answer(query, embeddings, qa_chain, instruction):
    try:
        # Подготовка запроса пользователя с учётом инструкции
        prepared_query = str(instruction) + ' ' + query
        # Поиск ответа на запрос
        answer = qa_chain(prepared_query, embeddings)
        return answer
    except Exception as e:
        return str(e)


Наконец можно задать вопросы нашей цепочке и получить правильные ответы!

In [ ]:
# Зададим 1вопрос пользователя
user_query = 'Возможно ли оформление результатов поверки нанесением знака поверки на самоклеящуюся этикетку (в том числе с помощью принтера), которая в дальнейшем наклеивается на средство измерений или вклеивается в паспорт?'
print(get_answer(user_query, embeddings, qa_chain, instruction))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'result': 'Да, возможно оформление результатов поверки нанесением знака поверки на самоклеящуюся этикетку (в том числе с помощью принтера), которая в дальнейшем наклеивается на средство измерений или вклеивается в паспорт. Это предусмотрено пунктом 4 Порядка проведения поверки средств измерений, требований к знаку поверки и содержанию свидетельства о поверке, утвержденного приказом Минпромторга России от 02.07.2015 № 1815.'}


In [ ]:
# Зададим 2 вопрос пользователя
user_query = 'Каков порядок представления на поверку средств измерений, изготовленных до 29.12.2021 и не имеющих заводских номеров?'
print(get_answer(user_query, embeddings, qa_chain, instruction))

{'result': 'Ответ:\nПорядок представления на поверку средств измерений, изготовленных до 29.12.2021 и не имеющих заводских номеров, определен в статье 13 Федерального закона от 26.06.2008 № 102-ФЗ "Об обеспечении единства измерений" (далее - Федеральный закон). Согласно части 4 статьи 13 Федерального закона, результаты поверки средств измерений подтверждаются сведениями о результатах поверки средств измерений, включенными в Федеральный информационный фонд по обеспечению единства измерений (далее - Фонд).\n\nВ соответствии с частью 6 статьи 13 Федерального закона, сведения о результатах поверки средств измерений передаются в Фонд проводящими поверку средств измерений юридическими лицами и индивидуальными предпринимателями в срок, установленный в порядке, предусмотренном частью 5 статьи 13 Федерального закона.\n\nСледовательно, для средств измерений, изготовленных до 29.12.2021 и не имеющих заводских номеров, необходимо предоставить информацию о результатах поверки в соответствии с устан

In [ ]:
# Зададим  3 запрос пользователя
user_query = 'Пунктом 3 Требований к содержанию свидетельства о поверке, утвержденных приказом Минпромторга России от 31.07.2020 № 2510, установлено, что свидетельство о поверке средства измерений и средства измерений, применяемого в качестве эталона единицы величины, должно содержать в том числе информацию о модификациях средства измерений (при наличии).Если в описании типа на средство измерений указаны не модификации средства измерений, а модели, диапазоны дозирования, классы точности, исполнения, то относится ли данная информация к модификации средств измерений?'
print(get_answer(user_query, embeddings, qa_chain, instruction))

{'result': 'В соответствии с пунктом 3 Требований к содержанию свидетельства о поверке, утвержденных приказом Минпромторга России от 31.07.2020 № 2510, в свидетельстве о поверке средства измерений должна содержаться информация о модификациях средства измерений (при наличии). Если в описании типа на средство измерений указаны не модификации средства измерений, а модели, диапазоны дозирования, классы точности, исполнения, то такая информация не может считаться модификацией средства измерений.'}
